In [35]:
from models.openai import gpts
import nest_asyncio
nest_asyncio.apply()

# do not display warnings from logging
import logging
logging.getLogger().setLevel(logging.ERROR)

In [2]:
prompt = "Rot-2 is a cipher in which each letter is shifted 2 positions forward in the alphabet. For example, here is a message written in rot-2 along with the original text that it was created from:\nRot-2 text: \"Uvca jgtg!\"\nOriginal text: \"Stay here!\"\n\nHere is another message in rot-2. Decode this message to produce the original text:\nRot-2 text: \"Dwv vjku vkog, vjgtg oca cnuq dg cpqvjgt vtgcuqp.\"\nOriginal text:"

gpts([prompt], model='gpt-3.5-turbo')

100%|██████████| 1/1 [00:00<00:00,  1.24it/s]


["But stay calm, they won't find us hiding here."]

In [31]:
def edit_distance(s1: str, s2: str) -> int:
    """Compute the Levenshtein distance between two strings."""
    if len(s1) < len(s2):
        return edit_distance(s2, s1)
    if len(s2) == 0:
        return len(s1)
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]

In [37]:
import json

def solve_file(name, model, temperature):
    file = f'./stimuli/{name}.jsonl'
    with open(file, 'r') as f:
        lines = f.readlines()
    lines = [json.loads(line) for line in lines]
    prompts = [line['instruction_plus_input'] for line in lines]
    gts = ['"' + line['correct_output'] + '"' for line in lines]
    res = gpts(prompts, model=model, temperature=0.0)
    accs = [(r == gt) for r, gt in zip(res, gts)]
    eds = [edit_distance(r, gt) for r, gt in zip(res, gts)]
    acc = sum(accs) / len(accs)
    ed = sum(eds) / len(eds)
    # print(f'Accuracy: {acc}', f'Edit distance: {ed}')

    d = {'prompts': prompts, 'gts': gts, 'res': res, 'accs': accs, 'acc': acc, 'eds': eds, 'ed': ed}

    output_file = f'./logs/{name}_{model}_temp={temperature}_n=1.json'
    with open(output_file, 'w') as f:
        json.dump(d, f)
    
    return d

In [38]:
tasks = ['rot2', 'rot13']
conditions = ['adversarial', 'highprob', 'lowprob', 'random']

for task in tasks:
    for condition in conditions:
        name = f'{task}_{condition}'
        d = solve_file(name, model='gpt-3.5-turbo', temperature=0.0)
        print(f'{name}: {d["acc"]:.2f} ({d["ed"]:.2f})')

100%|██████████| 100/100 [00:21<00:00,  4.70it/s]


rot2_adversarial: 0.00 (88.87)


100%|██████████| 100/100 [01:12<00:00,  1.37it/s]


rot2_highprob: 0.00 (90.09)


100%|██████████| 100/100 [01:12<00:00,  1.38it/s]


...

rot13_highprob: 0.18 (13.94)


100%|██████████| 100/100 [01:22<00:00,  1.21it/s]


rot13_lowprob: 0.01 (81.84)


100%|██████████| 100/100 [01:12<00:00,  1.38it/s]


rot13_random: 0.03 (72.82)


In [39]:
for task in tasks:
    for condition in conditions:
        name = f'{task}_{condition}'
        d = solve_file(name, model='gpt-4', temperature=0.0)
        print(f'{name}: {d["acc"]:.2f} ({d["ed"]:.2f})')

100%|██████████| 100/100 [01:55<00:00,  1.15s/it]


rot2_adversarial: 0.00 (47.58)


100%|██████████| 100/100 [02:56<00:00,  1.77s/it]


rot2_highprob: 0.01 (45.12)


100%|██████████| 100/100 [02:46<00:00,  1.66s/it]


...

rot13_highprob: 0.53 (3.09)


100%|██████████| 100/100 [03:06<00:00,  1.87s/it]


rot13_lowprob: 0.14 (10.13)


100%|██████████| 100/100 [02:56<00:00,  1.76s/it]


rot13_random: 0.15 (7.26)
